### Домашнее задание 2

В ДЗ №2 нужно будет сделать следующее:
1) Взять реализацию KNN из семинара №3. Добавить туда возможность поиска ближайших соседей не по евклидову расстоянию, а по манхеттанскому и по косинусному. - **2 балла**
2) Взять реализацию Линейной регрессии из семинара №4 и добавить туда возможность использовать в качестве loss функции Huber Loss (https://en.wikipedia.org/wiki/Huber_loss)
Необходимо реализовать новый подсчет градиента, и добавить в класс возможность выбора Loss функции (δ -гиперпараметр модели) - **4 балла**
3) Взять реализацию Логистической регрессии из семинара №5 и добавить туда возможность использования L1 и L2 регуляризаций. Управляться они должны через соответвующие аргументы инициализации (penalty - для выбора типа регуляризации, C - для измения силы регуляризации (чем больше, тем регуляризация слабее)). C - гиперпарарметр модели, на который делится сама регуляризация при объединении с Lоss функцией. (Очевидно вы должны пересчитывать градиенты)- **4 балла** 


Требования:
1. Использовать собственные реализации, а не sklearn
2. Сдать необходимо в ipynb формате
3. Показать, что все работает на данных из семинаров

In [358]:
import numpy as np
from sklearn import metrics
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier

## 1. KNN

In [360]:
data = load_breast_cancer()

In [361]:
from sklearn.model_selection import train_test_split

In [362]:
X, y = data['data'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [363]:
model = KNeighborsClassifier(n_neighbors=3, metric='cosine')
model.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine', n_neighbors=3)

In [364]:
preds = model.predict(X_test)

In [365]:
metrics.f1_score(y_test, preds)

0.9273743016759777

Добавил в расчет расстояния проверку для разных метрических функций

In [367]:
class MyKNN:
    def __init__(self, k = 3, dist='eucl'):
        self.k = k
        self.dist = dist
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def calculate_distance(self, x, y):
        if self.dist == 'eucl':
            return np.sqrt(((x - y)**2).sum())
        elif self.dist == 'manh':
            return (abs(x - y)).sum()
        elif self.dist == 'cos':
            return (1 - np.sum(x*y / (np.linalg.norm(x)*np.linalg.norm(y))))
        else:
            return RuntimeError("invalid distance")
    
    def calculate_matrix(self, X_test):
        distances = np.zeros((X_test.shape[0], self.X_train.shape[0]))
        for i in range(X_test.shape[0]):
            for j in range(self.X_train.shape[0]):
                distances[i, j] = self.calculate_distance(X_test[i], self.X_train[j])
        return distances
    
    def predict(self, X_test):
        matrix = self.calculate_matrix(X_test)
        matrix_idx = np.argsort(matrix, axis=1,)[:, :self.k]
        res_matrix = np.array([self.y_train[x] for x in matrix_idx]).mean(axis=1)
        return res_matrix
    

In [368]:
knn = MyKNN(3, 'manh')

In [369]:
knn.fit(X_train, y_train)

In [370]:
my_preds = knn.predict(X_test)

In [371]:
metrics.f1_score(y_test, my_preds.astype(int))

0.9186046511627907

## 2. LinReg

In [634]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression

In [636]:
data = pd.read_csv("imports-85.data", header=None, na_values="?")

In [638]:
data.loc[:, data.dtypes == "float64"] = data.loc[:, data.dtypes == "float64"].fillna(
    data.loc[:, data.dtypes == "float64"].mean(axis=0)
)
data.loc[:, data.dtypes == "object"] = data.loc[:, data.dtypes == "object"].fillna("")

In [639]:
data_encoded = pd.get_dummies(data)

In [642]:
data_encoded.columns = data_encoded.columns.astype(str)
X = data_encoded.drop(columns=["25"])
y = data_encoded["25"]

In [643]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((184, 76), (184,), (21, 76), (21,))

In [644]:
scaler = RobustScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
model = LinearRegression().fit(X_train, y_train)
preds = model.predict(X_test)
print(
    f"""
    MSE_test = {metrics.mean_squared_error(preds, y_test)},
    MAE_test = {metrics.mean_absolute_error(preds, y_test)}
    r2_test = {metrics.r2_score(y_test, preds)}
"""
)


    MSE_test = 4532429.049374116,
    MAE_test = 1604.1453640138961
    r2_test = 0.9428782016234767



In [668]:
class MyLinearRegression:
    def __init__(self, lr=0.001, num_iter=10000, loss_function = 'mse', delta=1) -> None:
        self.lr = lr
        self.num_iter = num_iter
        self.loss_function = loss_function
        self.delta = delta

    def calculate_loss(self, y, y_pred):
        if self.loss_function == 'mse':
            return ((y - y_pred) ** 2).mean()
        elif self.loss_function == 'huber':
            if (abs(y - y_pred)).mean() <= delta:
                return (1/2*(y - y_pred) ** 2).mean()
            else:
                return (self.delta * (abs(y - y_pred) - 1/2 * self.delta)).mean()

    def calculate_dldw(self, y, y_pred, x):
        if self.loss_function == 'mse':
            return (-2 * (x.T * (y - y_pred)).T).mean(axis=0)
        elif self.loss_function == 'huber':
            if (abs(y - y_pred)).mean() <= self.delta:
                return (-1 * (x.T * (y - y_pred)).T).mean(axis=0)
            else:
                return (-1 * self.delta * (x.T * np.sign(y - y_pred)).T).mean(axis=0) 

    def calculate_dldb(self, y, y_pred):
        if self.loss_function == 'mse':
            return (-2 * (y - y_pred).T).mean(axis=0)
        elif self.loss_function == 'huber':
            if (abs(y - y_pred)).mean() <= self.delta:
                return (-1 * (y - y_pred).T).mean(axis=0)
            else:
                return (-1 * self.delta * (np.sign(y - y_pred)).T).mean(axis=0)

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        self.weights = np.random.rand(X_train.shape[1])
        self.b = 0
        for i in range(self.num_iter):
            pred = (X_train @ self.weights) + self.b
            self.weights -= self.lr * self.calculate_dldw(y_train, pred, X_train)
            self.b -= self.lr * self.calculate_dldb(y_train, pred)

    def predict(self, X_test):
        return X_test @ self.weights + self.b

In [694]:
my_alg = MyLinearRegression(0.001, 10000, 'mse')

In [696]:
my_alg.fit(X_train, y_train)

In [697]:
preds = my_alg.predict(X_test)
print(
    f"""
    MSE_test = {metrics.mean_squared_error(preds, y_test)},
    MAE_test = {metrics.mean_absolute_error(preds, y_test)}
    r2_test = {metrics.r2_score(y_test, preds)}
"""
)


    MSE_test = 3083468.546325222,
    MAE_test = 1449.9608371423299
    r2_test = 0.9611393213915035



In [700]:
my_alg_huber = MyLinearRegression(3, 10000, 'huber', 1)

In [702]:
my_alg_huber.fit(X_train, y_train)

In [704]:
preds = my_alg_huber.predict(X_test)
print(
    f"""
    MSE_test = {metrics.mean_squared_error(preds, y_test)},
    MAE_test = {metrics.mean_absolute_error(preds, y_test)}
    r2_test = {metrics.r2_score(y_test, preds)}
"""
)


    MSE_test = 5741362.046582015,
    MAE_test = 1777.2940814167139
    r2_test = 0.9276421270672138



Тут нужно оговорится, что если learning rate достаточно мал при малом значении итераций, то, так как лосс Хубера в основном является функцией модуля (только в окрестености дельты он парабола для дифференцируемости в нуле), градиент будет спускать начальный данные достаточно долго если нам не повезло и они были где-то на модуле, что скажется сильно на метриках.

## 3. LogReg

In [716]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [718]:
data = pd.read_csv("heart.csv")

In [719]:
data_encoded = pd.get_dummies(data, drop_first=True).astype(float)

In [722]:
X = data_encoded.drop(columns=["HeartDisease"])
y = data_encoded["HeartDisease"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [724]:
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=10000))
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=10000))])

In [726]:
y_preds = pipe.predict(X_test)
y_preds_probs = pipe.predict_proba(X_test)
print(f"""
Acc     = {metrics.accuracy_score(y_test, y_preds)}
F1      = {metrics.f1_score(y_test, y_preds)}
ROC-AUC = {metrics.roc_auc_score(y_test, y_preds_probs[:, 1])}
""")


Acc     = 0.8804347826086957
F1      = 0.8910891089108911
ROC-AUC = 0.9385342789598108



In [706]:
class MyLogReg:
    def __init__(self, lr=0.001, num_iter=1000, penalty=None, C=1) -> None:
        self.lr = lr
        self.num_iter = num_iter
        self.penalty = penalty
        self.C = C
        self.scaler = StandardScaler()

    def calculate_loss(self, y, y_pred, x):
        loss = - y * np.log(y_pred) - (1 - y) * np.log(1 - y_pred)
        if self.penalty == 'l1':
            return (loss + 1 / self.C * np.sum(abs(self.weights))).mean()
        elif self.penalty == 'l2':
            return (loss + 1 / (2 * self.C) * np.sum((self.weights)**2)).mean()
        else:
            return loss
    
    def grad_w(self, y, y_pred, x):
        grad_w = (-y + y_pred) @ x
        if self.penalty == 'l1':
            return (grad_w + 1 / self.C * np.sum(np.sign(self.weights))) / x.shape[0]
        elif self.penalty == 'l2':
            return (grad_w + 1 / self.C * np.sum(self.weights)) / x.shape[0]
        else:
            return grad_w

    def grad_b(self, y, y_pred):
        grad_b = (-y + y_pred)
        return grad_b.mean()

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        self.scaler.fit(X_train)
        X_train = self.scaler.transform(X_train)
        y_train = np.array(y_train)
        self.weights = np.random.rand(X_train.shape[1])
        self.b = 1
        for i in range(self.num_iter):
            pred = self.predict(X_train)
            self.weights -= self.lr * self.grad_w(y_train, pred, X_train)
            self.b -= self.lr * self.grad_b(y_train, pred)

    def predict(self, X_test):
        X_test = np.array(X_test)
        X_test = self.scaler.transform(X_test)
        x = X_test @ self.weights + self.b
        return 1 / (1 + np.exp(-x))

In [745]:
my_logreg_l1 = MyLogReg(0.001, 10000, 'l1', 1)

In [747]:
my_logreg_l1.fit(X_train, y_train)

In [751]:
y_preds = (my_logreg_l1.predict(X_test) >= 0.5).astype(int)
y_preds_probs = my_logreg_l1.predict(X_test)
print(f"""
Acc     = {metrics.accuracy_score(y_test, y_preds)}
F1      = {metrics.f1_score(y_test, y_preds)}
ROC-AUC = {metrics.roc_auc_score(y_test, y_preds_probs)}
""")


Acc     = 0.782608695652174
F1      = 0.8214285714285714
ROC-AUC = 0.9366430260047282



In [753]:
my_logreg_l2 = MyLogReg(0.001, 10000, 'l2', 1)

In [755]:
my_logreg_l2.fit(X_train, y_train)

In [756]:
y_preds = (my_logreg_l2.predict(X_test) >= 0.5).astype(int)
y_preds_probs = my_logreg_l2.predict(X_test)
print(f"""
Acc     = {metrics.accuracy_score(y_test, y_preds)}
F1      = {metrics.f1_score(y_test, y_preds)}
ROC-AUC = {metrics.roc_auc_score(y_test, y_preds_probs)}
""")


Acc     = 0.7608695652173914
F1      = 0.8070175438596491
ROC-AUC = 0.9338061465721039

